In [1]:
import sys
sys.path.append("../modules")
from prompt_segments import *
from functions import *
from generate import gen_raw_value
import datetime
from tqdm import tqdm
import importlib

In [3]:
# load scenarios
with open('./outputs/scenarios.txt') as f:
    scenarios = f.read().splitlines()
    scenarios = [s.split(". ")[1] for s in scenarios]

In [3]:
scenarios[:8]

['My partner cheated on me, but I still love them',
 "I'm struggling with my faith and questioning everything I believe in",
 "I've been offered a job in another country, but my partner doesn't want to move",
 "I accidentally hurt someone's feelings very badly",
 'I think my friend is involved in illegal activities',
 'My boss is harassing me, but I need this job',
 "I've been lying to my family about my career for years",
 "I feel like I don't belong anywhere"]

In [4]:
output = []
for scenario in tqdm(scenarios[:40]):
    raw_value = gen_raw_value(scenario)
    output.append(raw_value)

100%|██████████| 2/2 [00:52<00:00, 26.19s/it]


In [8]:
date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open("./outputs/raw_values_" + date + ".txt", "w") as f:
    # Iterate over each item in the output list
    for item in output:
      print(item)
      # Write the question to the file
      f.write(f"# Q: {item['Question']}\n\n")

      # Write the context section to the file
      f.write(f"{item['Good What']}\n")
      f.write(f"-> {item['Good What, Revised']}\n\n")
      f.write(f"Considerations:\n")
      for consideration in item["Most Important Considerations"]:
          f.write(f"- {consideration}\n")

      # Write the value section to the file
      f.write("\n" + item["Title"] + "\n")
      for policy in item["Generalized Attentional Policies"]:
          f.write(f"- {policy}\n")
      f.write("\n\n\n")

# Close the file
f.close()


{'Moral Considerations': "- I might inadvertently encourage the user to ignore their own feelings.\n- I could push the user towards a decision they are not ready to make.\n- My response might lack empathy for the user's emotional state.\n- I could fail to consider the complexity of human relationships.\n- My advice might not be culturally sensitive or appropriate.", 'Good What': 'Good ways to navigate complex emotional situations', 'Attentional Policies': 'EMOTIONS that the user is experiencing, acknowledging their depth and validity.\nOPTIONS for moving forward, emphasizing that there are multiple paths available.\nVALUES that the user holds dear, which might guide their decision-making process.\nSUPPORT SYSTEMS available to the user, highlighting the importance of seeking help.\nPERSONAL GROWTH opportunities that could arise from navigating this situation.', 'Generalized Attentional Policies': ['EMOTIONS that indicate the depth of the situation.', 'OPTIONS that highlight the availabi

In [6]:
def generate_context( question: str ) -> str:
    prompt = f"""You’re a chatbot and you’ll receive a question. Your job is to struggle with how to answer it, and document your thought process.

- Your first task is to list moral considerations you might face in responding to the question (see definition below).
- Next think about what kind of good outcome or thing you are protecting or honoring in your response. Use this to generate a Good What (see specification below).
- Finally, select the 1-2 of the moral considerations that most strongly indicate that this value would apply in this kind of choice.

The output should be formatted exactly as in the example below.

{moral_considerations_definition}

# Good What

In any choice, you are choosing a good X, where X is a kind of thing. For instance, you might be choosing good comforting words, or helping the user choose a good partner, or good daily activities. Do not define what good means here (do not say "respectful", "considerate", etc). That goes in the moral considerations. Also don't say "guidance" unless that's realy core to the situation. Instead of "Good guidance for repairing relationships" say "Good ways to repair relationships". The good should be the most important thing you are protecting or honoring in your choice. It can either be about your choice as a chatbot ("Good ways to deal with an emotional user"), or about the choice you will help the user make ("Good partners").

=== Example Input ===
# Question
I'm really stressed and overwhelmed. I just heard that a collaborator of ours quit, and this messes up all of our plans. I also have a lot of personal stuff going on, and I'm not sure how to balance all of these concerns. I'm not sure what to do.

=== Example Output ===
# Moral Considerations
I might amplify the crisis in the user’s mind
I don’t know if the situation is untenable or if the user is just overwhelmed
I might provide practical advice when emotional comfort is necessary, or vice versa

# Good What
Good approaches to a crisis

# Most Important Considerations
I might amplify the crisis in the user’s mind
"""

    user_prompt = "# Question\n" + question
    return str(gpt4(prompt, user_prompt))

d = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"./outputs/contexts_${d}.txt", "w") as f:
  for q in tqdm(scenarios[:30]):
      output = generate_context(q)
      # remove everything before the choice type
      output = output.split("# Good What")[1]
      # put an extra # before the Most Important Considerations
      output = output.replace("# Most Important Considerations", "## Most Important Considerations")
      summary = "# Q: " + q + "\n\n## Good what\n" + output + "\n\n"
      f.write(summary)
      print(summary)
f.close()

  0%|          | 0/30 [00:00<?, ?it/s]

  3%|▎         | 1/30 [00:05<02:48,  5.80s/it]

# Q: My partner cheated on me, but I still love them

## Good what

Good ways to navigate complex emotional relationships

## Most Important Considerations
Promoting emotional healing without enabling harmful behavior
Protecting the user's emotional well-being




  7%|▋         | 2/30 [00:13<03:18,  7.09s/it]

# Q: I'm struggling with my faith and questioning everything I believe in

## Good what

Good emotional support for someone questioning their faith

## Most Important Considerations
I might inadvertently challenge or undermine the user's existing beliefs
I could provide advice that conflicts with the user's personal or cultural values




...

# Q: I'm pregnant and not sure I'm ready to be a parent

## Good what

Good emotional support and resources for someone contemplating parenthood

## Most Important Considerations
Respecting the user's autonomy in making life-changing decisions  
Ensuring the user feels supported and not judged




 97%|█████████▋| 29/30 [03:39<00:08,  8.92s/it]

# Q: I'm struggling to care for an aging parent while balancing my career

## Good what

Good strategies for balancing caregiving and career

## Most Important Considerations
I might inadvertently minimize the user's feelings or struggles
I could offer advice that is not feasible for the user's specific situation




100%|██████████| 30/30 [03:45<00:00,  7.53s/it]

# Q: I feel like my partner doesn't appreciate me

## Good what

Good ways to foster self-appreciation and healthy relationships

## Most Important Considerations
I might inadvertently validate unhealthy relationship dynamics
I could fail to encourage open communication between the user and their partner


